In [1]:
from datetime import datetime
import os
import time
import requests
import json

import pickle
from pathlib import Path
import traceback

In [39]:
# call API to retrieve comments of a game with a game id

# game_steamid = 1716740          # starfield
# game_name = 'starfield'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

# game_steamid = 730
# game_name = 'counter-strike_2'

# game_steamid = 570
# game_name = "dota2"

# game_steamid = 2138330
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1814540
# game_name = 'cosmos_conquer'

# game_steamid = 1814520
# game_name = 'find_the_cat'

# game_steamid = 1814480
# game_name = 'fourplay_chess'

# game_steamid = 1757970
# game_name = 'oasis_dark_forest'

# game_steamid = 1607720
# game_name = 'rimebeard'

# game_steamid = 1814460
# game_name = 'the_brave_vs_dragon'

game_steamid = 1814510
game_name = 'yavi'

params = {
    'json':1,
    'cursor':'*',
    'num_per_page':100,
    'language':'english',       # only english reviews
    'filter':'recent',
    'review_type':'all',
    'purchase_type':'all'

}

# api doc: https://partner.steamgames.com/doc/store/getreviews
# reviews_req = requests.get(f"https://store.steampowered.com/appreviews/{game_steamid}", params=params)

In [40]:
# load the reviews from folder

reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'./{game_name}')
if game_folder.exists():
    try:
        latest_file_path = sorted(game_folder.glob('steam_reviews_*.pkl'))[-1]
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            cursor_str = reviews_reqs[-1]['cursor']
            print('Loaded:', latest_file_path)
            print('Cursor:', cursor_str)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()

Loaded: yavi/steam_reviews_1814510_20240311164621.pkl
Cursor: AoJwrazJ2v0CftXBkwM=


In [41]:
reviews_reqs[0]

{'success': 1,
 'query_summary': {'num_reviews': 15,
  'review_score': 6,
  'review_score_desc': 'Mostly Positive',
  'total_positive': 11,
  'total_negative': 4,
  'total_reviews': 15},
 'reviews': [{'recommendationid': '150188804',
   'author': {'steamid': '76561198239664996',
    'num_games_owned': 227,
    'num_reviews': 115,
    'playtime_forever': 242,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 242,
    'last_played': 1700070063},
   'language': 'english',
   'review': "Neat pokemon-like game, it's really fun and the monsters are really cool and interesting. It feels polished but there are some aspects that yet feel unfinished, overall i'd say it's worth it to give it a try if you are into this type of games",
   'timestamp_created': 1700081335,
   'timestamp_updated': 1700081335,
   'voted_up': True,
   'votes_up': 0,
   'votes_funny': 0,
   'weighted_vote_score': 0,
   'comment_count': 0,
   'steam_purchase': False,
   'received_for_free': True,
   'written_dur

In [42]:
len(reviews_reqs)

1

In [43]:
review_list = []

# datetime_until = datetime(2023, 1, 1, 0, 0, 0)

for reviews_req in reviews_reqs:
    review_list.extend(reviews_req['reviews'])

# filter by unique 'recommendationid'
unique_review_list = []
unique_review_ids = set()

for review in review_list:
    if review['recommendationid'] in unique_review_ids:
        continue

    # if review['timestamp_created'] < datetime_until.timestamp():
    #     continue

    unique_review_list.append(review)
    unique_review_ids.add(review['recommendationid'])

len(unique_review_list)

15

Very similar to the time I read the number of reviews: 96741, at 20240211:000030

In [44]:
# save the unique reviews

with open(f'./{game_name}/steam_reviews_{game_steamid}_unique.pkl', 'wb') as f:
    pickle.dump(unique_review_list, f)

The number excludes off-topic reviews (??)

---

In [24]:
reviews_req.json()

{'success': 1,
 'query_summary': {'num_reviews': 100},
 'reviews': [{'recommendationid': '153833346',
   'author': {'steamid': '76561198170136267',
    'num_games_owned': 343,
    'num_reviews': 24,
    'playtime_forever': 2595,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 2595,
    'last_played': 1694489855},
   'language': 'english',
   'review': 'This is the worst parts of bethesda games without the best parts of bethesda games',
   'timestamp_created': 1702867677,
   'timestamp_updated': 1702867677,
   'voted_up': False,
   'votes_up': 7,
   'votes_funny': 0,
   'weighted_vote_score': '0.540235579013824463',
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': False,
   'hidden_in_steam_china': True,
   'steam_china_location': ''},
  {'recommendationid': '153833339',
   'author': {'steamid': '76561197991612208',
    'num_games_owned': 2661,
    'num_reviews': 40,
    'playtime_forever': 852,
    'playtime_

In [6]:
with open('reviews.json', 'w') as f:
    json.dump(reviews_req.json(), f, indent=2)

In [4]:
print(reviews_req.json()['reviews'][2]['review'])

Was initially disappointed with this game but its come along way and I seriously enjoy it. Cant wait for CK to be added since modding it right now is a massive pain
